In [1]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *
import gym
import matplotlib.pyplot as plt
from FrozenLake import *

#np.set_printoptions(precision=4)
import math
from utils import *

nrows = 20
nS = nrows**2
nA = 4
seed = get_current_seed()
seed = 44697628841978080856580175700798794719
gamma = .9
tau = 1.
env = FrozenLakeEnv(is_slippery=False, seed=seed, desc=generate_random_map(nrows))#, render_mode="human")
xi_frozen = np.ones(env.nS) * 1/env.nS
env.reset()
xi = np.ones(nS) * 1/nS
tmdp = TMDP(env, xi_frozen, tau=tau, gamma=gamma, seed=seed)


Current seed for result reproducibility: 53166883074729279014800260307620810660


In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class PolicyPi(nn.Module):
    def __init__(self, nS, nA, hidden_dim=64):
        super().__init__()
        self.nS = nS
        self.nA = nA
        # Embedding layer to handle integer states
        self.embedding = nn.Embedding(nS, nS)
        self.hidden = nn.Linear(nS, hidden_dim)
        self.classify = nn.Linear(hidden_dim, nA)

    def forward(self, s):
        # Embed the integer state
        s = self.embedding(s)
        s = F.relu(self.hidden(s))
        logits = self.classify(s)
        return logits
    
    def act(self, s):
        with torch.no_grad():
            # Preparing the input state tensor
            s = torch.tensor([s], dtype=torch.long).to(device)  # Wrap integer state in a list, convert to tensor
            logits = policy_pi(s)
            logits = logits.squeeze(dim=0)
            probs = F.softmax(logits, dim=-1)
            action = torch.multinomial(probs, num_samples=1)
            return action.item()
    
    def get_probabilities(self):
        probs = np.zeros((nS, nA))
        with torch.no_grad():
            for s in range(nS):
                s = torch.tensor([s], dtype=torch.long).to(device)
                logits = policy_pi(s)
                logits = logits.squeeze(dim=0)
                probs[s] = F.softmax(logits, dim=-1).cpu().numpy()
        return probs
    
    def get_logits(self):
        full_logits = np.zeros((nS, nA))
        with torch.no_grad():
            for s in range(nS):
                s = torch.tensor([s], dtype=torch.long).to(device)
                logits = policy_pi(s)
                logits = logits.squeeze(dim=0)
                full_logits[s] = logits.cpu().numpy()
        return full_logits
    
policy_pi = PolicyPi(nS, nA).to(device)

In [3]:
opt = torch.optim.Adam(policy_pi.parameters(), lr=1e-3)

In [4]:
def curriculum_PG(tmdp:TMDP, policy_pi:PolicyPi, opt, episodes=5000, alpha=.25, batch_nS=1, biased=True):
    
    nS, nA = tmdp.env.nS, tmdp.env.nA
    done = False 
    terminated = False

    batch_size = 0
    # State action next-state value function
    U = np.zeros((nS, nA, nS))
    V = np.zeros(nS)
    Q = np.zeros((nS, nA))

    # Hyperparameters decay
    dec_alpha = alpha
    
    # Curriculum parameters
    alpha_star = dec_alpha
    tau_star = 1
    convergence_t = 0

    # Mid-term results
    rewards_records = []
    states = []
    actions = []
    rewards = []
    flags_list = []
    t = 0
    for episode in range(episodes): # Each episode is a single time step
        
        while True:
            s = tmdp.env.s
            states.append(s)
            a = policy_pi.act(s) # Select action from policy

            s_prime, r, flags, p =  tmdp.step(a)

            flags["terminated"] = terminated
            actions.append(a)
            rewards.append(r)
            flags_list.append(flags)

            # Reset the environment if a terminal state is reached or if a teleportation happened
            if flags["done"]:# or flags["teleport"]:
                tmdp.reset()
                batch_size += 1

            if episode < episodes-1: # move to next time step
                break   
            else: # if reached the max num of time steps, wait for the end of the trajectory for consistency
                print("Ending the loop")
                terminated = True
                flags["terminated"] = terminated
                break # temporary ending condition To be Removed
                if flags["done"]:
                    done = True
                    break

        # Processing the batch
        if( (batch_size != 0 and batch_size % batch_nS == 0) or done or terminated):
            # Extract previous policy for future comparison
            
            cum_reward = np.zeros_like(rewards)
            reward_len = len(rewards)
            for i in reversed(range(reward_len)):
                cum_reward[i] = rewards[i] + tmdp.gamma*cum_reward[i+1] if i < reward_len-1 else rewards[i]
                
            logits_old = policy_pi.get_logits()
            pi_old = policy_pi.get_probabilities()

            tensor_states = torch.tensor(states, dtype=torch.long).to(device)
            tensor_actions = torch.tensor(actions, dtype=torch.long).to(device)
            tensor_cum_reward = torch.tensor(cum_reward, dtype=torch.float).to(device)
            opt.zero_grad()
            logits = policy_pi(tensor_states)
            # Calculate negative log probability (-log P) as loss.
            # Cross-entropy loss is -log P in categorical distribution.

            logs_prob = -F.cross_entropy(logits, tensor_actions, reduction='none')
            loss = -logs_prob * tensor_cum_reward
            loss.sum().backward()
            opt.step()

            # Learning the value function
            for i in range(len(states)):
                t += 1
                dec_alpha= max(1e-5, alpha*(1 - t/episodes))
                s = states[i]
                a = actions[i]
                s_prime = states[i+1] if i < len(states)-1 else states[i]
                a_prime = actions[i+1] if i < len(states)-1 else pi_old[s].argmax()
                r = rewards[i]
                flags = flags_list[i]

                td_error = dec_alpha*(r + tmdp.gamma*V[s_prime] - V[s])
                V[s] = V[s] + dec_alpha*td_error
                U[s,a,s_prime] = U[s,a,s_prime] + dec_alpha*(r + tmdp.gamma*V[s_prime] - U[s,a,s_prime])
                Q[s,a] = Q[s,a] + dec_alpha*(r + tmdp.gamma*Q[s_prime, a_prime] - Q[s,a])

            # Bound evaluation
            if( tmdp.tau > 0):
                pi = policy_pi.get_probabilities()
                rel_pol_adv = compute_relative_policy_advantage_function(pi, pi_old, Q)
                rel_model_adv = compute_relative_model_advantage_function(tmdp.env.P_mat, tmdp.xi, U)
                d = compute_d_from_tau(tmdp.env.mu, tmdp.env.P_mat, tmdp.xi, pi_old, tmdp.gamma, tmdp.tau)
                delta = compute_delta(d, pi_old)
                pol_adv = compute_expected_policy_advantage(rel_pol_adv, d)
                model_adv = compute_expected_model_advantage(rel_model_adv, delta)

                delta_U = get_sup_difference_U(U)
                if delta_U == 0:
                    delta_U = (tmdp.env.reward_range[1]-tmdp.env.reward_range[0])/(1-tmdp.gamma)
                    
                d_inf_pol = get_d_inf_policy(pi, pi_old)
                d_inf_model = get_d_inf_model(tmdp.env.P_mat, tmdp.xi)
                d_exp_pol = get_d_exp_policy(pi, pi_old, d)
                d_exp_model = get_d_exp_model(tmdp.env.P_mat, tmdp.xi, delta)

                # Compute optimal values
                optimal_pairs = get_teleport_bound_optimal_values(pol_adv, model_adv, delta_U,
                                                                d_inf_pol, d_exp_pol, d_inf_model,
                                                                d_exp_model, tmdp.tau, tmdp.gamma, biased=biased)
                teleport_bounds = []
                for alpha_prime, tau_prime in optimal_pairs:
                    bound = compute_teleport_bound(alpha_prime, tmdp.tau, tau_prime, pol_adv, model_adv,
                                                    tmdp.gamma, d_inf_pol, d_inf_model,
                                                    d_exp_pol, d_exp_model, delta_U, biased=biased)
                    teleport_bounds.append(bound)
                
                alpha_star, tau_star = get_teleport_bound_optima_pair(optimal_pairs, teleport_bounds)

                print(optimal_pairs)
                print(teleport_bounds)
                print("Updating the policy with alpha_star: ", alpha_star, "tau_star: ",tau_star)
                tmdp.update_tau(tau_star)
    
                if tau_star == 0:
                    print("Converged to the original problem, episode {}".format(episode))
                    convergence_t = episode
                    # Set lambda parameter for eligibility traces for the fine tuning of the policy
            r_sum = sum(rewards)
            print("Running episode {} reward {}".format(episode, r_sum))
            # Reset the batch
            states = []
            actions = []
            rewards = []
            flags_list = []
            batch_size = 0
            rewards_records.append(r_sum)

    return {"Q": Q, "V": V, "U": U, "policy_pi": policy_pi, "convergence_t": convergence_t, "rewards": rewards_records}



In [17]:
tmdp.reset()
tmdp.update_tau(.5)
cur_res = curriculum_PG(tmdp, policy_pi, opt, episodes=300000, alpha=.25, batch_nS=1, biased=False)

KeyboardInterrupt: 

: 

In [6]:
res = bellman_optimal_q(tmdp.env.P_mat, tmdp.env.reward, tmdp.gamma)
Q = res["Q"]

d = compute_d_from_tau(tmdp.env.mu, tmdp.env.P_mat, tmdp.xi, get_policy(Q), tmdp.gamma, 0.)
d_curr = compute_d_from_tau(tmdp.env.mu, tmdp.P_mat_tau, tmdp.xi, get_policy(policy_pi.get_probabilities()), tmdp.gamma, 0.)

print(get_policy(Q) - get_policy(policy_pi.get_probabilities()))

r_s_a = compute_r_s_a(tmdp.env.P_mat, tmdp.env.reward)

j_opt = compute_j(r_s_a, get_policy(Q), d, tmdp.gamma)
j_curr = compute_j(r_s_a, get_policy(policy_pi.get_probabilities()), d_curr, tmdp.gamma)
print("optimal performance: ",j_opt, "curriculum performance: ",j_curr)

[[ 0.  1.  0. -1.]
 [ 0.  1. -1.  0.]
 [ 0.  0.  0.  0.]
 ...
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0. -1.  0.]]
optimal performance:  0.013302794647291147 curriculum performance:  0.0


In [16]:
Q = cur_res["Q"]
print(Q)

[[1.26462815e-03 4.66352560e-03 4.57481569e-03 1.38917348e-07]
 [1.35067704e-02 2.80340682e-03 6.01054024e-03 5.47126938e-03]
 [4.05905334e-03 3.09667151e-03 4.59847107e-03 0.00000000e+00]
 ...
 [1.36251175e-03 9.90257673e-08 7.50414788e+00 1.00774296e-03]
 [0.00000000e+00 0.00000000e+00 8.67017216e+00 0.00000000e+00]
 [4.52946214e-03 5.32592422e-03 3.05887965e-03 3.88646212e-03]]


In [14]:
tmdp = TMDP(env, xi_frozen, tau=0., gamma=gamma, seed=seed)
pi = get_policy(Q)
env.render_mode = "human"
tmdp.reset()
done = False
step = 0
while True:
    a = greedy(tmdp.env.s, pi, tmdp.env.allowed_actions[int(tmdp.env.s)])
    s_prime, reward, flags, prob = tmdp.step(a)
    if flags["done"]:
        tmdp.reset()
        break
    step +=1
    if step > max(100,nrows*2):
        break